# 프로젝트 정보
**개요**
- 사회 이슈를 발견하고 문제점을 해결하기 위한 데이터 분석을 진행한다.
- api로 데이터 수집, 분석 툴을 활용한 데이터 분석, 대시보드를 활용한 데이터 시각화를 진행한다.
- 데이터 분석을 통한 최적의 문제 해결방안을 제시한다.

**배경**
- 최근 전기차 업체들이 경쟁적으로 전기차 신차를 출시하기 시작하면서 전기차가 빠르게 대중화되는 반면, 전기차 충전 인프라가 전기차 보급 속도를 따라가지 못하는 상황으로 일부 지역에서는 전기차 충전기의 이용과 점유를 두고 충전 인프라에 대한 불편을 호소하고 있다.
- 서울시는 기후변화대응 종합계획(2022-2026)에서 10대 핵심사업 중 하나로 전기차 10% 시대 실현을 발표하며 ‘생활권 5분 충전망’ 구축을 목표하고있다. 이에 따라 효율적인 전기차 충전 인프라 확충을 위해 최적의 입지를 분석하여 선정한다.

**주제**
- 서울시 신규 전기차 충전소 부지 선정

**설명** 
- 분석 배경 및 필요성
- 데이터 수집
- 데이터 전처리 및 분석
    - 충전기 설치 완료 구역 분석
    - 충전기 수요지 분석
    - 설치 제한구역 확인
    - 충전기 설치 수량 분석
    - 최적 입지 지역 선정
- 시각화 결과물
- 결론

---
# 데이터 정리

전국 공공 및 민간 충전기 운영현황 정보
- https://safemap.go.kr/opna/data/dataView.do

In [ ]:
import requests
import pandas as pd 


serviceKey = "V9RHJ29A-V9RH-V9RH-V9RH-V9RHJ29AHU"  # 일반 인증키 (Decoding)

url = 'http://safemap.go.kr/openApiService/data/getChargingStationData.do'


params ={'serviceKey' : serviceKey, 'numOfRows' : '20000', 'pageNo' : '1', 'dataType' : 'XML', 'CTPRVN_CD' : '11' }
#numOfRows가 23000는 불러와지지 않아서 20000개까지만 호출

response = requests.get(url, params=params)

#print(response.content)  # bytes 형태여서 보기 힘듦.
content = response.content.decode('utf-8')  # utf-8로 디코딩함. 
print(content)

chager_place_df = pd.read_xml(response.content, xpath=".//item")  

In [57]:
#데이터 프레임 확인
display(chager_place_df.head(3)) 

#서울시 (CTPRVN_CD=11)만 호출되었는지 확인
print(chager_place_df['CTPRVN_CD'].unique())

#csv 파일로 저장
chager_place_df.to_csv('서울시 충전소 정보.csv', index = False)

,OBJT_ID,STAT_NM,STAT_ID,CHGER_ID,CHGER_TY,USE_TM,BUSI_ID,BUSI_NM,TELNO,ADRES,RN_ADRES,CTPRVN_CD,SGG_CD,EMD_CD,X,Y
0,1,신교공영주차장,ST111106,3,2,24시간 이용가능,ST,에스트래픽,1566-1704,NaN,서울특별시 종로구 자하문로 89 (지하 2층 주차장),11,11110,11110102,1.413423e+07,4.520828e+06
1,2,신교공영주차장,ST111106,4,2,24시간 이용가능,ST,에스트래픽,1566-1704,NaN,서울특별시 종로구 자하문로 89 (지하 2층 주차장),11,11110,11110102,1.413423e+07,4.520828e+06
2,3,신교공영주차장,ST111106,1,4,24시간 이용가능,ST,에스트래픽,1566-1704,NaN,서울특별시 종로구 자하문로 89 (지하 2층 주차장),11,11110,11110102,1.413423e+07,4.520828e+06


[11]


### 서울시 교통량 이력 정보(사용X)
- https://data.seoul.go.kr/dataList/OA-13316/A/1/datasetView.do
- 지정하는 날짜와 시간에 대해서만 조회할 수 있음(사용X)

In [ ]:
import requests
import pandas as pd 


serviceKey = "64706171756b736b3530424a445873"  # 일반 인증키 (Decoding)

url = 'http://openapi.seoul.go.kr:8088/64706171756b736b3530424a445873/xml/VolInfo/1/5/A-05/20240301/20/'


params ={'KEY' : serviceKey, 'TYPE' : 'xml', 'SERVICE' : 'VolInfo', 'START_INDEX' : '1', 'END_INDEX' : '1000'}
#numOfRows가 23000는 불러와지지 않아서 20000개까지만 호출

response = requests.get(url, params=params)

#print(response.content)  # bytes 형태여서 보기 힘듦.
content = response.content.decode('utf-8')  # utf-8로 디코딩함. 
print(content)

traffic_df = pd.read_xml(response.content)  

### 서울시 교통량
- https://topis.seoul.go.kr/refRoom/openRefRoom_2.do
- '지점코드별', '월별', '유입' 교통량 합계
- '서울시 교통량 지점정보'의 지점코드로 위치(경도, 위도) 표시

In [276]:
#월별 데이터 불러오기
df_202402 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/02월 서울시 교통량 조사자료(2024).xlsx', sheet_name='2024년 02월')
df_202401 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/01월 서울시 교통량 조사자료(2024).xlsx', sheet_name='2024년 01월')
df_202312 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/12월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 12월')
df_202311 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/11월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 11월')
df_202310 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/10월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 10월')
df_202309 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/09월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 09월')
df_202308 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/08월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 08월')
df_202307 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/07월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 07월')
df_202306 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/06월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 06월')
df_202305 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/05월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 05월')
df_202304 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/04월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 04월')
df_202303 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/03월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 03월')
df_202302 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/02월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 02월')
df_202301 = pd.read_excel('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/01월 서울시 교통량 조사자료(2023).xlsx', sheet_name='2023년 01월')

In [293]:
# '지점별', '유입' 기준의 교통량 합계 데이터프레임 함수
def get_traffic(df) : 
    df = df[df.loc[:,'방향'] == '유입']
    df['합계'] = df.loc[:,'0시':'23시'].sum(axis=1)
    df = pd.DataFrame(df.groupby(['지점명','지점번호']).agg({'합계' : 'sum'}))
    return df 


In [ ]:
# 월별 교통량 합계 데이터프레임 병합
# 월별 날짜로 컬럼명 수정
month_traffic = get_traffic(df_202402).rename(columns={'합계' : '2024-02'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202401), on = ['지점명', '지점번호']).rename(columns={'합계' : '2024-01'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202312), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-12'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202311), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-11'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202310), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-10'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202309), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-09'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202308), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-08'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202307), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-07'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202306), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-06'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202305), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-05'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202304), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-04'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202303), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-03'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202302), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-02'})
month_traffic = pd.merge(month_traffic, get_traffic(df_202301), on = ['지점명', '지점번호']).rename(columns={'합계' : '2023-01'})
month_traffic

### 서울시 교통량 지점 정보
- https://data.seoul.go.kr/dataList/OA-13314/A/1/datasetView.do
- 각 교통지점에 대한 위치정보 확인

In [312]:
import requests
import pandas as pd 

serviceKey = "64706171756b736b3530424a445873"  # 일반 인증키 (Decoding)

url = 'http://openapi.seoul.go.kr:8088/64706171756b736b3530424a445873/xml/SpotInfo/1/1000/'


params ={'KEY' : serviceKey, 'TYPE' : 'xml', 'SERVICE' : 'SpotInfo', 'START_INDEX' : '1', 'END_INDEX' : '10000'}
#numOfRows가 23000는 불러와지지 않아서 20000개까지만 호출

response = requests.get(url, params=params)

#print(response.content)  # bytes 형태여서 보기 힘듦.
content = response.content.decode('utf-8')  # utf-8로 디코딩함. 
print(content)

traffic_point_df = pd.read_xml(response.content)  
traffic_point_df

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><SpotInfo><list_total_count>139</list_total_count><RESULT><CODE>INFO-000</CODE><MESSAGE>정상 처리되었습니다</MESSAGE></RESULT><row><spot_num>A-01</spot_num><spot_nm>성산로(금화터널)</spot_nm><grs80tm_x>195489</grs80tm_x><grs80tm_y>452136</grs80tm_y></row><row><spot_num>A-02</spot_num><spot_nm>사직로(사직터널)</spot_nm><grs80tm_x>196756.776106</grs80tm_x><grs80tm_y>452546.638644</grs80tm_y></row><row><spot_num>A-03</spot_num><spot_nm>자하문로(자하문터널)</spot_nm><grs80tm_x>197216.855046</grs80tm_x><grs80tm_y>454350.990432</grs80tm_y></row><row><spot_num>A-04</spot_num><spot_nm>대사관로(삼청터널)</spot_nm><grs80tm_x>198648.893154</grs80tm_x><grs80tm_y>455200.108465</grs80tm_y></row><row><spot_num>A-05</spot_num><spot_nm>율곡로(안국역)</spot_nm><grs80tm_x>198645.671347</grs80tm_x><grs80tm_y>452937.216603</grs80tm_y></row><row><spot_num>A-06</spot_num><spot_nm>창경궁로(서울여자대학교)</spot_nm><grs80tm_x>199825.89671</grs80tm_x><grs80tm_y>453668.322568</grs80tm_y></row><row><spot_num>A-07</

,list_total_count,CODE,MESSAGE,spot_num,spot_nm,grs80tm_x,grs80tm_y
0,139.0,None,None,None,None,NaN,NaN
1,NaN,INFO-000,정상 처리되었습니다,None,None,NaN,NaN
2,NaN,None,None,A-01,성산로(금화터널),195489.000000,452136.000000
3,NaN,None,None,A-02,사직로(사직터널),196756.776106,452546.638644
4,NaN,None,None,A-03,자하문로(자하문터널),197216.855046,454350.990432
...,...,...,...,...,...,...,...
136,NaN,None,None,F-06,경부고속도로,202107.000000,443264.000000
137,NaN,None,None,F-07,분당수서로,207716.000000,444241.000000
138,NaN,None,None,F-08,강남순환로(관악터널),191832.000000,437667.000000
139,NaN,None,None,F-09,서부간선지하도로,189706.000000,441044.000000


In [311]:
# month_traffic_df의 지점번호(spot_num) 컬럼명 변경
traffic_point_df = traffic_point_df.rename(columns = {'spot_num' : '지점번호'})

# 지점번호 기준으로 병합 (지점병 위치정보))
# 필요없는 컬럼 삭제
month_traffic_df = pd.merge(month_traffic, traffic_point_df, on = '지점번호').drop(columns = ['list_total_count', 'CODE', 'MESSAGE', 'spot_nm'])
month_traffic_df.head(3) 

,지점번호,2024-02,2024-01,2023-12,2023-11,2023-10,2023-09,2023-08,2023-07,2023-06,2023-05,2023-04,2023-03,2023-02,2023-01,grs80tm_x,grs80tm_y
0,C-03,1555465.0,1668646.0,1715164.0,1744913.0,1716376.0,1498958.0,1501420.0,1732890.0,1732920.0,1681605.0,1746870.0,1826804.0,1611171.0,1641095.0,187636.0,452084.0
1,D-35,624039.0,772619.0,766336.0,776765.0,749099.0,747214.0,772550.0,763077.0,742412.0,753886.0,739451.0,782502.0,711953.0,724579.0,202677.0,443638.0
2,D-43,984935.0,1072936.0,1060556.0,1073123.0,1109828.0,1054499.0,1115710.0,1122308.0,1103407.0,1130594.0,1101811.0,1141937.0,1007466.0,1057444.0,201776.0,446166.0
